In [ ]:
import geopandas as gpd 
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Import the voronoi shape of India
import dssg.dataio.osm_to_voronoi_mapping as ovm
data_dir = os.environ.get("DATA_DIR")
india_voronoi_gpd = gpd.read_file(data_dir + "voronoi3_clip/voronoi3_clip.shp")

In [ ]:
india_voronoi_gpd.head()

In [ ]:
import dssg.dataio.osm_data_extraction as ode
india_shape = os.environ.get("DATA_DIR") + "/gadm36_shp/gadm36_IND_2.shp"

In [ ]:
%%prun -s cumulative -q -l 10 -T ../profile/extract_district_dataframe
import matplotlib.pyplot as plt 
%matplotlib inline
india_gpd = gpd.read_file(india_shape)
#Extract a district from gadm36_IND_2.shp file
araria_gdf = ode.extract_district_dataframe(india_gpd, 'Araria')

In [ ]:
print(open('../profile/extract_district_dataframe', 'r').read())

In [ ]:
%%prun -s cumulative -q -l 10 -T ../profile/extract_district_voronoi_clipped
#Extract the GeoDataFrame of the voronoi clipped to the district boundary
araria_voronoi_gpd_clipped = ovm.extract_district_voronoi_clipped(india_voronoi_gpd, araria_gdf)

In [ ]:
print(open('../profile/extract_district_voronoi_clipped', 'r').read())

In [ ]:
ovm.plot_district_voronoi(araria_voronoi_gpd_clipped, "Araria")

In [ ]:
araria_voronoi_gpd_clipped.head()

In [ ]:
#%%prun -s cumulative -q -l 10 -T ../profile/extract_district_voronoi
#araria_voronoi_gpd = ovm.extract_district_voronoi(india_voronoi_gpd, araria_gdf)

In [ ]:
#print(open('../profile/extract_district_voronoi', 'r').read())

In [ ]:
#araria_voronoi_gpd.head()

In [ ]:
#ovm.plot_district_voronoi(araria_voronoi_gpd, "Araria")

In [ ]:
%%prun -s cumulative -q -l 10 -T ../profile/extract_district_voronoi_wo_clipping
araria_voronoi_joined = ovm.extract_district_voronoi_wo_clipping(india_voronoi_gpd, araria_gdf)

In [ ]:
araria_voronoi_joined.head()

In [ ]:
print(open('../profile/extract_district_voronoi_wo_clipping', 'r').read())

In [ ]:

ovm.plot_district_voronoi(araria_voronoi_joined, "Araria")

In [ ]:
#Write the district voronoi geodataframe to a file
ovm.write_district_voronoi_to_shapefile(araria_voronoi_joined, os.environ.get("DATA_DIR") + "/voronoi3_clip/araria-voronoi.shp")

In [ ]:
# Extract OSM dataframe based on voronoi boundary of the district to prepare to get osm data.
from shapely.geometry import mapping, Polygon
import osmnx as ox 
# Date based extraction of polygon and graph
cs = '[out:json][timeout:180][date:"2021-03-01T00:00:00Z"]'
(araria_voronoi_poly, araria_voronoi_graph) = ode.create_knots_and_edges_from_boundary(araria_voronoi_joined, cs)


In [ ]:
import osmnx as ox
araria_voronoi_kefig, araria_voronoi_ax = ox.plot_graph(araria_voronoi_graph)
plt.tight_layout()

In [ ]:
tags = {'amenity':True, 'building':True, 'emergency':True, 'highway':True, 'footway':True, 'landuse': True, 'water': True}
araria_voronoi_osmdf = ode.extract_osm_csv(araria_voronoi_poly, tags)

In [ ]:
araria_voronoi_osmdf.head()

In [ ]:
araria_voronoi_osmdf.shape

In [ ]:
print(type(araria_voronoi_osmdf))

In [ ]:
araria_voronoi_osmdf.to_csv(os.environ.get("OSM_DIR") + "/araria_voronoi_osm.csv")

In [ ]:
#How to read the araria_voronoi_osm csv file into a geodataframe
from shapely import wkt
import pandas as pd 
araria_voronoi_osm_df = pd.read_csv(os.environ.get("OSM_DIR") + "/araria_voronoi_osm.csv", low_memory=False)
araria_voronoi_osm_df['geometry'] = araria_voronoi_osm_df['geometry'].apply(wkt.loads)
araria_voronoi_osm_gdf = gpd.GeoDataFrame(araria_voronoi_osm_df, geometry='geometry')

In [ ]:
import pandas as pd 
osm_dhs_df = pd.DataFrame(columns=['OSMID', 'DHSID', 'DHSCLUST', 'DISTRICTID'])
osm_dhs_df.head()

In [ ]:
pt_count = 0
poly_count = 0
ln_str_count = 0
mp_count = 0
for index, row in araria_voronoi_osm_gdf.iterrows():
    if row.geometry.geom_type == 'Point':
        pt_count += 1
        for ind, ro in araria_voronoi_joined.iterrows():
            if ro.geometry.contains(row.geometry):
                osm_dhs_df = osm_dhs_df.append({'OSMID':row.osmid, 'DHSID':ro.DHSID, 'DHSCLUST': ro.DHSCLUST, 'DISTRICTID': ro.index_right},    ignore_index=True)
                break
    elif row.geometry.geom_type == 'Polygon':
        poly_count += 1
        for ind, ro in araria_voronoi_joined.iterrows():
            if ro.geometry.contains(row.geometry.centroid):
                osm_dhs_df = osm_dhs_df.append({'OSMID':row.osmid, 'DHSID':ro.DHSID, 'DHSCLUST': ro.DHSCLUST, 'DISTRICTID': ro.index_right}, ignore_index=True)
                break
    elif row.geometry.geom_type == 'LineString':
        ln_str_count += 1
    elif row.geometry.geom_type == 'MultiPolygon':
        mp_count += 1

print("Total Pts = ", pt_count)
print("Total Polylines = ", ln_str_count)
print("Total polygons = ", poly_count)
print("Total Multi Polygons = ", mp_count)

In [ ]:
osm_dhs_df.shape

In [ ]:
print(osm_dhs_df)

In [ ]:
osm_dhs_df.to_csv(os.environ.get("OSM_DIR") + "/araria-osm-dhs.csv")

In [ ]:
fig = plt.figure(num=None, figsize=(20,20))
ax = araria_voronoi_joined['geometry'].boundary.plot()
araria_voronoi_osmdf['geometry'].plot(ax=ax, linewidth=1, color='red', markersize=10, marker='o')